In [ ]:
import json
import sys
import os
from pymongo import MongoClient
import io
from bson import ObjectId
import wget
import subprocess
import shutil
import os
import sys
import re

In [ ]:
def is_files_in_list(my_list):
    regexp_first = re.compile(r'good(G2B|B2G)')
    res = [x for x in my_list if re.search(regexp_first, x)] 
    
    if len(res) > 0:
        return True
    else:
        return False

In [ ]:
path = "/obfuscated/DATASETS/SARD/000/"

files = []

regexp = re.compile(r'_\d{2}[a-z].+[a-z]')

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    in_files = is_files_in_list(f)
    if in_files == True:
        pass
    else:
        for file in f:
            if not regexp.search(file) and not '_bad.' in file and not '_good1.' in file and not '.DS_Store' in file:
                files.append(os.path.join(r, file))

In [ ]:
print(str(len(files)))

In [ ]:
raw_lst = [r'(helperBad|badVaSink[BG]?)|(helperGood(G2B|B2G)?\d*)|(good(G2B|B2G)?\d*VaSink[BG]?)',r'badSource(_[a-z])?|badSink(_[a-z])?|good(G2B\d*|B2G\d*)?Source(_[a-z])?|good(G2B\d*|B2G\d*)?Sink(_[a-z])?']
reg_lst = []

for raw_regex in raw_lst:
    reg_lst.append(re.compile(raw_regex))

In [ ]:
for file in files:
    
    with open(file,'r') as f:
        
        a = f.read()
        
        if any(compiled_reg.search(a) for compiled_reg in reg_lst):
            pass
        else:
            files_second_filter.append(os.path.join(r, file)) 

In [ ]:
print(str(len(files_second_filter)))

In [ ]:
for element in files_second_filter:
    print(element)

In [ ]:
for element in files_second_filter:
    my_basic_string = element
    info_string = my_basic_string.split('SARD', 1)[1]
    
    id = info_string
    path_separation = info_string.split('CWE',1)
    path = path_separation[0]
    file_name = 'CWE'+path_separation[1]
    cwe = file_name.split('__',1)[0]
    functional_variant_name = file_name.split('__',1)[1].split('.',1)[0].rsplit('_', 1)[0]
    flow_variant = file_name.split('__',1)[1].split('.',1)[0].rsplit('_', 1)[1]
    language = file_name.split('__',1)[1].split('.',1)[1]
    
    print(id)
    print(path)
    print(file_name)
    print(cwe)
    print(functional_variant_name)
    print(flow_variant)
    print(language)

In [ ]:
client = MongoClient('mongodb://obfuscated/')
db = client['obfuscated']
collection = db["obfuscated"]

my_doc = None;

for element in files_second_filter:
    my_basic_string = element
    info_string = my_basic_string.split('SARD', 1)[1]
    
    identification = info_string
    path_separation = info_string.split('CWE',1)
    path = path_separation[0]
    file_name = 'CWE'+path_separation[1]
    cwe = file_name.split('__',1)[0]
    functional_variant_name = file_name.split('__',1)[1].split('.',1)[0].rsplit('_', 1)[0]
    flow_variant = file_name.split('__',1)[1].split('.',1)[0].rsplit('_', 1)[1]
    language = file_name.split('__',1)[1].split('.',1)[1]
    
    my_doc = { "id":identification, "path":path ,"file_name": file_name, "cwe": cwe, "functional_variant_name":functional_variant_name, "flow_variant":flow_variant, "language":language }
        
    try:
        collection.insert_one(my_doc)
    except Exception:
        pass
        
    my_doc = None;